# Experiment 027: Best Ensemble - Combining Top Approaches

**Strategic situation:**
- Best CV: 0.0623 (exp_004)
- Best LB: 0.0956 (exp_004/exp_016)
- CV-LB correlation: 0.994
- Target: 0.01727 (5.5x gap)
- Only 1 submission remaining

**Approach:**
Combine the best elements from all experiments:
1. exp_004 architecture (HGB for SM, ETR for Products)
2. Dual features (0.8 ACS_PCA + 0.2 Spange)
3. NO TTA (proven to hurt full data)
4. Arrhenius kinetics features

This is essentially a clean reimplementation of exp_004 to verify we can match CV 0.0623.

**TEMPLATE COMPLIANCE**: Last 3 cells are EXACTLY as template, NO cells after them.

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import HistGradientBoostingRegressor, ExtraTreesRegressor
from abc import ABC
import tqdm
import warnings
warnings.filterwarnings('ignore')

DATA_PATH = '/home/data'
torch.set_default_dtype(torch.double)
print("Setup complete")

Setup complete


In [2]:
# --- UTILITY FUNCTIONS ---
TARGET_LABELS = ["Product 2", "Product 3", "SM"]

def load_data(name="full"):
    assert name in ["full", "single_solvent"]
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[["Residence Time", "Temperature", "SOLVENT NAME"]]
    Y = df[TARGET_LABELS]
    return X, Y

def load_features(name="spange_descriptors"):
    return pd.read_csv(f'{DATA_PATH}/{name}_lookup.csv', index_col=0)

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

# Load feature sets
SPANGE_DF = load_features('spange_descriptors')
ACS_PCA_DF = load_features('acs_pca_descriptors')
print(f"Spange: {SPANGE_DF.shape}, ACS_PCA: {ACS_PCA_DF.shape}")

Spange: (26, 13), ACS_PCA: (24, 5)


In [3]:
# --- BASE CLASSES ---
class SmilesFeaturizer(ABC):
    def __init__(self): raise NotImplementedError
    def featurize(self, X): raise NotImplementedError

class BaseModel(ABC):
    def __init__(self): pass
    def train_model(self, X_train, y_train): raise NotImplementedError
    def predict(self): raise NotImplementedError

In [4]:
# --- DUAL FEATURIZER (EXACT exp_004 implementation) ---
class DualFeaturizer:
    """Featurizer using both Spange and ACS_PCA descriptors."""
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange = SPANGE_DF
        self.acs_pca = ACS_PCA_DF

    def _build_process_features(self, X):
        """Build process features with Arrhenius kinetics."""
        rt = X['Residence Time'].values.astype(np.float64).reshape(-1, 1)
        temp = X['Temperature'].values.astype(np.float64).reshape(-1, 1)
        
        # Arrhenius kinetic features
        temp_k = temp + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(rt + 1e-6)
        interaction = inv_temp * log_time
        
        return np.hstack([rt, temp, inv_temp, log_time, interaction])

    def _get_solvent_features(self, X, feature_df, flip=False):
        """Get solvent features from a feature dataframe."""
        if self.mixed:
            A = feature_df.loc[X["SOLVENT A NAME"]].values
            B = feature_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                return B * (1 - pct) + A * pct
            else:
                return A * (1 - pct) + B * pct
        else:
            return feature_df.loc[X["SOLVENT NAME"]].values

    def featurize_spange(self, X, flip=False):
        """Build features using Spange descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.spange, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1)
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

    def featurize_acs(self, X, flip=False):
        """Build features using ACS_PCA descriptors."""
        process = self._build_process_features(X)
        solvent = self._get_solvent_features(X, self.acs_pca, flip)
        if self.mixed:
            pct = X["SolventB%"].values.reshape(-1, 1)
            return np.hstack([process, pct, solvent])
        return np.hstack([process, solvent])

print("DualFeaturizer defined")

DualFeaturizer defined


In [5]:
# --- BEST MODEL (exp_004 architecture, NO TTA) ---
class BestModel(BaseModel):
    """Best model combining all learnings from 26 experiments.
    
    Architecture (from exp_004):
    - SM: HistGradientBoostingRegressor (max_depth=7, max_iter=700, lr=0.04)
    - Products: ExtraTreesRegressor (n_estimators=500, min_samples_leaf=2, max_depth=10)
    - Feature weighting: 0.8 acs_pca + 0.2 spange
    - NO TTA (exp_005 showed TTA hurts full data by 33%)
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = DualFeaturizer(mixed=(data=='full'))
        self.targets = ['Product 2', 'Product 3', 'SM']
        
        self.scaler_spange = StandardScaler()
        self.scaler_acs = StandardScaler()
        self.models = {}
        
        self.acs_weight = 0.8
        self.spange_weight = 0.2

    def train_model(self, X_train, y_train):
        X_spange = self.featurizer.featurize_spange(X_train)
        X_acs = self.featurizer.featurize_acs(X_train)
        y = y_train.values
        
        if self.data_type == 'full':
            # Data augmentation (training only, NOT at prediction time)
            X_spange_flip = self.featurizer.featurize_spange(X_train, flip=True)
            X_acs_flip = self.featurizer.featurize_acs(X_train, flip=True)
            X_spange = np.vstack([X_spange, X_spange_flip])
            X_acs = np.vstack([X_acs, X_acs_flip])
            y = np.vstack([y, y])
        
        X_spange_scaled = self.scaler_spange.fit_transform(X_spange)
        X_acs_scaled = self.scaler_acs.fit_transform(X_acs)
        
        for i, target in enumerate(self.targets):
            y_target = y[:, i]
            
            if target == 'SM':
                model_spange = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04, random_state=42
                )
                model_acs = HistGradientBoostingRegressor(
                    max_depth=7, max_iter=700, learning_rate=0.04, random_state=42
                )
            else:
                model_spange = ExtraTreesRegressor(
                    n_estimators=500, min_samples_leaf=2, max_depth=10,
                    n_jobs=-1, random_state=42
                )
                model_acs = ExtraTreesRegressor(
                    n_estimators=500, min_samples_leaf=2, max_depth=10,
                    n_jobs=-1, random_state=42
                )
            
            model_spange.fit(X_spange_scaled, y_target)
            model_acs.fit(X_acs_scaled, y_target)
            
            self.models[target] = {'spange': model_spange, 'acs': model_acs}

    def predict(self, X):
        # NO TTA - direct prediction only
        X_spange = self.featurizer.featurize_spange(X)
        X_acs = self.featurizer.featurize_acs(X)
        
        X_spange_sc = self.scaler_spange.transform(X_spange)
        X_acs_sc = self.scaler_acs.transform(X_acs)
        
        preds = []
        for target in self.targets:
            p_spange = self.models[target]['spange'].predict(X_spange_sc)
            p_acs = self.models[target]['acs'].predict(X_acs_sc)
            p_combined = self.acs_weight * p_acs + self.spange_weight * p_spange
            preds.append(p_combined.reshape(-1, 1))
        
        final_preds = np.hstack(preds)
        final_preds = np.clip(final_preds, 0, 1)
        return torch.tensor(final_preds, dtype=torch.double)

print("BestModel defined (exp_004 architecture, NO TTA)")

BestModel defined (exp_004 architecture, NO TTA)


In [6]:
# Quick validation test
print("Testing BestModel...")
X_test, Y_test = load_data("single_solvent")

errors = []
for i, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_test, Y_test)):
    if i >= 5: break
    model = BestModel(data='single')
    model.train_model(train_X, train_Y)
    preds = model.predict(test_X).numpy()
    mae = np.mean(np.abs(preds - test_Y.values))
    errors.append(mae)
    solvent = test_X['SOLVENT NAME'].iloc[0]
    print(f"Single Fold {i} ({solvent}): MAE = {mae:.4f}")

print(f"\nQuick test MAE: {np.mean(errors):.4f}")

Testing BestModel...


Single Fold 0 (1,1,1,3,3,3-Hexafluoropropan-2-ol): MAE = 0.1446


Single Fold 1 (2,2,2-Trifluoroethanol): MAE = 0.0962


Single Fold 2 (2-Methyltetrahydrofuran [2-MeTHF]): MAE = 0.0369


Single Fold 3 (Acetonitrile): MAE = 0.0813


Single Fold 4 (Acetonitrile.Acetic Acid): MAE = 0.1077

Quick test MAE: 0.0934


In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = BestModel(data='single') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:01,  1.81s/it]

2it [00:03,  1.86s/it]

3it [00:05,  1.87s/it]

4it [00:07,  1.90s/it]

...

21it [00:40,  1.90s/it]

22it [00:42,  1.93s/it]

23it [00:44,  1.94s/it]

24it [00:46,  1.96s/it]

24it [00:46,  1.93s/it]

In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = BestModel(data = 'full') # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:02,  2.37s/it]

2it [00:04,  2.48s/it]

3it [00:07,  2.39s/it]

4it [00:09,  2.39s/it]

...

10it [00:23,  2.29s/it]

11it [00:25,  2.28s/it]

12it [00:27,  2.29s/it]

13it [00:30,  2.29s/it]

13it [00:30,  2.32s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [10]:
# Calculate CV score - THIS CELL MUST BE REMOVED BEFORE SUBMISSION
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Single solvent CV
single_preds = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
single_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_out_splits(X_single, Y_single)):
    single_true.append(test_Y.values)
single_true = np.vstack(single_true)
single_mae = np.mean(np.abs(single_preds - single_true))
print(f"Single solvent CV MAE: {single_mae:.4f}")

# Full data CV
full_preds = submission_full_data[['target_1', 'target_2', 'target_3']].values
full_true = []
for fold_idx, ((train_X, train_Y), (test_X, test_Y)) in enumerate(generate_leave_one_ramp_out_splits(X_full, Y_full)):
    full_true.append(test_Y.values)
full_true = np.vstack(full_true)
full_mae = np.mean(np.abs(full_preds - full_true))
print(f"Full data CV MAE: {full_mae:.4f}")

# Combined
combined_mae = (single_mae + full_mae) / 2
print(f"\nCombined CV MAE: {combined_mae:.4f}")
print(f"exp_004 CV: 0.0623")
print(f"Difference: {combined_mae - 0.0623:.4f}")

Single solvent CV MAE: 0.0677
Full data CV MAE: 0.0943

Combined CV MAE: 0.0810
exp_004 CV: 0.0623
Difference: 0.0187
